In [ ]:
import pretrainedmnist

net = pretrainedmnist.mnist(pretrained=True)

# NB: 
At the end, ANOTHER version, trained myself, with pytorch (experiment on runge kutta)

In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets


In [ ]:
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((28,28)),
#     transforms.Grayscale(),
     transforms.ToTensor()])
#,
 #    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
from scipy.misc import imread, imsave
import io
import sqlite3
def make_connection(p):
    return sqlite3.connect(p)
def preload_all_queries(conn, more=''):
    c = conn.cursor()
    c.execute('''SELECT id_a,student,* FROM capture_zone WHERE type=4 '''+more+''' ORDER BY student,id_a,id_b ASC''')
    res = {}
    for r in c.fetchall():
        k = r[1]
        if not (k in res):
            res[k] = []
        r = r[:-1] + (imread(io.BytesIO(r[-1])),)
        #import pytesseract
        #print(pytesseract.image_to_string(imread(io.BytesIO(r[-1])), config='--psm 10'))
        # ^ doesn't seem to work too well on this data
        #import base64
        #r = r[:-1] + (base64.b64encode(r[-1]).decode('ascii'),)
        #im = imread(io.BytesIO(r[-1]))
        res[k].append(r)
    return res


In [ ]:
info = preload_all_queries(make_connection('test3/capture.sqlite'), more=' AND student='+str(42))

In [ ]:
from matplotlib import pyplot as plt
#im = info[42][2][13]
#im = np.transpose(info[42][2][13], [0,1,2])
#im = np.array([np.array(info[42][i][13]) for i in range(10)])
im = np.array([transform(info[42][i][13]).numpy()[0,:,:] for i in range(20)]) 
#print(im.shape)
#im = transform(im)
#plt.imshow(im[0,:,:])
#print(im.min(), im.max(), im.shape)
#im = (im - np.mean(im)) / np.std(im)

#transform(im.shape)
with torch.no_grad():
    print(im.size)
    pred = net(torch.from_numpy(im))
    print(pred.shape)
    print(np.argmax(pred, axis=1), pred)
    


In [ ]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((28,28)),
        #transforms.Resize((50,50)),
        #transforms.CenterCrop((28,28)),
        #transforms.Grayscale(),
        transforms.ToTensor()
        ])


In [ ]:
im = info[42][81][13]
im0 = im
print(im.shape)
im = np.transpose(transform(im0).numpy(), [1,2,0])
print(im.shape)
R = im[:,:,0]
G = im[:,:,1]
B = im[:,:,2]
#im = np.sum(im, axis=2) / 3 #, np.array(im[:,:,1:2])# - np.sum(im, axis=2, keepdims=True))
#im[((B*2-G-R)>.05)] = np.max(im)
#im[((R*2-G-B)>.05)] = np.max(im)

im = np.sum(im, axis=2) / 3 #, np.array(im[:,:,1:2])# - np.sum(im, axis=2, keepdims=True))
im = (im - np.min(im)) / (np.max(im) - np.min(im))

im[((B*2-G-R)>.05)] = np.max(im)
im[((R*2-G-B)>.05)] = np.max(im)
im = 1 - im

print(im.shape)
#plt.imshow(im)
plt.imshow(im, cmap='gray')
#im = transform(im[:,:,0]).numpy()
print(np.min(im), np.max(im))
im2 = im

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


net = torch.load('model.torch')

In [ ]:

transform2 = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.Resize((28,28)),
    transforms.Resize((28,28)),
    transforms.CenterCrop((28,28)),
    #transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

print(im0.shape, im2.shape)
im = np.array((im2[...,np.newaxis]*np.ones((1,1,3)))*255, dtype=np.uint8)
print(im.shape, np.min(im), np.max(im))
batch = transform2(im)
batch = batch[np.newaxis, 1:2, ::, ::]
print(batch.shape, torch.min(batch), torch.max(batch))


#print("batch", batch.shape, batch.numpy().shape)
plt.imshow(batch.numpy()[0,0,:,:])

print(batch.min(), batch.max())
##batch = batch.transpose(0,1)
#print(batch.shape)
#batch = batch.numpy()#.sum(keepdims = True)
#batch = torch.from_numpy(batch[np.newaxis])
#print(batch.shape)
with torch.no_grad():
    pred = net(batch)
print(np.argmax(pred), pred)